In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

In [3]:
import svox

In [4]:
import csrc as C_

In [5]:
tree = svox.N3Tree(data_dim=3)
tree.to("cuda:0")

svox.N3Tree(N=2, data_dim=3, depth_limit=10, capacity:1/1, data_format:RGBA)

In [6]:
indices = torch.tensor([[0.25, 0.25, 0.25],
                        [0.25, 0.25, 0.75],
                        [0.25, 0.75, 0.25],
                        [0.25, 0.75, 0.75],
                        [0.75, 0.25, 0.25],
                        [0.75, 0.25, 0.75],
                        [0.75, 0.75, 0.25],
                        [0.75, 0.75, 0.75]]).cuda()
values = torch.tensor([[1, 1, 1],
                       [1, 1, 2],
                       [1, 2, 1],
                       [1, 2, 2],
                       [2, 1, 1],
                       [2, 1, 2],
                       [2, 2, 1],
                       [2, 2, 2]]).float()
tree.set(indices, values)

In [7]:
tree[0], tree[1], tree[2], tree[3], tree[4], tree[5], tree[6], tree[7]

(N3TreeView(tensor([1., 1., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 1., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 2., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 2., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 1., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 1., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 2., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 2., 2.], device='cuda:0', grad_fn=<SelectBackward0>)))

In [8]:
tree._all_leaves()

tensor([[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [0, 1, 1, 1]])

In [9]:
sel = (*tree._all_leaves()[:1].T,)
print(sel)
tree.refine(sel=sel)

(tensor([0]), tensor([0]), tensor([0]), tensor([0]))


True

In [10]:
tree.data

Parameter containing:
tensor([[[[[1., 1., 1.],
           [1., 1., 2.]],

          [[1., 2., 1.],
           [1., 2., 2.]]],


         [[[2., 1., 1.],
           [2., 1., 2.]],

          [[2., 2., 1.],
           [2., 2., 2.]]]],



        [[[[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]]],


         [[[0., 0., 0.],
           [0., 0., 0.]],

          [[0., 0., 0.],
           [0., 0., 0.]]]]], device='cuda:0', requires_grad=True)

In [17]:
q_indices = torch.tensor([
    [0.125, 0.125, 0.125],
    [0.125, 0.125, 0.375],
    [0.125, 0.375, 0.125],
    [0.125, 0.375, 0.375],
    [0.375, 0.125, 0.125],
    [0.375, 0.125, 0.375],
    [0.375, 0.375, 0.125],
    [0.375, 0.375, 0.375],
    
    [0.125, 0.65, 0.125],
    [0.125, 0.55, 0.125],
    [0.125, 0.51, 0.125],
    [0.125, 0.50, 0.125],
    [0.125, 0.49, 0.125],
    [0.125, 0.45, 0.125],
    [0.125, 0.39, 0.125],
    
    [0.25, 0.25, 0.75],
    [0.25, 0.75, 0.25],
    [0.25, 0.75, 0.75],
    [0.75, 0.25, 0.25],
    [0.75, 0.25, 0.75],
    [0.75, 0.75, 0.25],
    [0.75, 0.75, 0.75]
]).cuda()
out = C_.query_interp(tree._spec(world=True), q_indices)
out

tensor([[1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000],
        [0.5625, 1.0125, 0.5625],
        [0.5625, 0.9000, 0.5625],
        [0.5625, 0.8550, 0.5625],
        [0.5625, 0.8438, 0.5625],
        [0.5400, 0.5400, 0.5400],
        [0.7000, 0.7000, 0.7000],
        [0.9400, 0.9400, 0.9400],
        [1.0000, 1.0000, 2.0000],
        [1.0000, 2.0000, 1.0000],
        [1.0000, 2.0000, 2.0000],
        [2.0000, 1.0000, 1.0000],
        [2.0000, 1.0000, 2.0000],
        [2.0000, 2.0000, 1.0000],
        [2.0000, 2.0000, 2.0000]], device='cuda:0')

In [16]:
q_indices = torch.tensor([[0.26, 0.75, 0.26], [0.8, 0.8, 0.8]]).cuda()
C_.query_interp(tree._spec(world=True), q_indices[0:1])

tensor([[1.0200, 2.0000, 1.0200]], device='cuda:0')

In [27]:
tree[0], tree[1], tree[2], tree[3], tree[4], tree[5], tree[6], tree[7], tree[8]

(N3TreeView(tensor([1., 1., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 2., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 2., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 1., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 1., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 2., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([2., 2., 2.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 1., 1.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([1., 1., 1.], device='cuda:0', grad_fn=<SelectBackward0>)))

In [28]:
tree._all_leaves().shape

torch.Size([15, 4])

In [27]:
indices = torch.tensor([[0.0, 0.0, 0.0], [1., 1., 1.]]).cuda()
values = torch.tensor([[3, 3, 3], [6, 6, 100000]]).float()
tree.set(indices, values)

In [25]:
tree[0], tree[1], tree[2], tree[3], tree[4], tree[5], tree[6], tree[7]

(N3TreeView(tensor([3., 3., 3.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([6.0000e+00, 6.0000e+00, 1.0000e+05], device='cuda:0',
        grad_fn=<SelectBackward0>)))

(N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([0., 0., 0.], device='cuda:0', grad_fn=<SelectBackward0>)),
 N3TreeView(tensor([6.0000e+00, 6.0000e+00, 1.0000e+05], device='cuda:0',
        grad_fn=<SelectBackward0>)))

In [13]:
q_indices = torch.tensor([[0.26, 0.75, 0.26], [0.8, 0.8, 0.8]]).cuda()
C_.query_interp(tree._spec(world=True), q_indices[0:1])

tensor([[2.4000e-03, 2.4000e-03, 4.0000e+01]], device='cuda:0')

In [19]:
C_.query_vertical(tree._spec(world=True), indices)

(tensor([[3.0000e+00, 3.0000e+00, 3.0000e+00],
         [6.0000e+00, 6.0000e+00, 1.0000e+05]], device='cuda:0'),
 tensor([0, 7], device='cuda:0'))